<a href="https://colab.research.google.com/github/veeralakrishna/END/blob/main/Session%2010/Neural_Machine_Translation_by_Jointly_Learning_to_Align_and_Translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Neural Machine Translation by Jointly Learning to Align and Translate

In [5]:
# !cat /proc/cpuinfo
# !cat /proc/meminfo

In [6]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-8d7578f4-764d-1c1f-7093-0539ff882721)


In [7]:
!pip install torchtext==0.6.0

     |████████████████████████████████| 71kB 5.8MB/s 
     |████████████████████████████████| 1.2MB 12.8MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [8]:
import torch
import torch.nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import os
import math
import time
import random


from tqdm.notebook import tqdm

In [9]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
seed_everything(42)

In [10]:
!python -m spacy download de

     |████████████████████████████████| 14.9MB 800kB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907057 sha256=bbd7e1ab5bce21b4eb5f6c8aa898d3fcc1c877441a83d032a3fcf00a60e878d0
  Stored in directory: /tmp/pip-ephem-wheel-cache-06m0scme/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [11]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [12]:
def tokenize_de(text):
  return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
  return [tok.text for tok in spacy_en.tokenizer(text)]

In [13]:
SRC = Field(
    tokenize = tokenize_en,
    init_token = '<sos>',
    eos_token = '<eos>',
    lower = True
)

TRG = Field(
    tokenize = tokenize_de,
    init_token = '<sos>',
    eos_token = '<eos>',
    lower = True
)

In [14]:
train_data, valid_data, test_data = Multi30k.splits(exts=('.en', '.de'), fields=(SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:02<00:00, 515kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 166kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 165kB/s]


In [15]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

In [16]:
BATCH_SIZE = 256
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device
)

Device: cuda


In [17]:
class Encoder(torch.nn.Module):

  def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
    super().__init__()
    self.embedding = torch.nn.Embedding(input_dim, emb_dim)
    self.rnn = torch.nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
    self.fc = torch.nn.Linear(enc_hid_dim * 2, dec_hid_dim)
    self.dropout = torch.nn.Dropout(dropout)

  def forward(self, src):

    # src -> [src len, batch size]

    embedded = self.dropout(self.embedding(src))
    # embedded -> [src len, batch size, emb_dim]

    outputs, hidden = self.rnn(embedded)
    # outputs -> [src len, batch size, enc_hid_dim * num_directions]
    #         -> [src len, batch size, enc_hid_dim * 2]
    # hidden -> [num_layers * num_directions, batch size, enc_hid_dim]
    #        -> [2, batch size, enc_hid_dim]

    concatenated = torch.cat((hidden[-2, :, :], hidden[-1, :, :]),dim=1)
    # concatenated -> [batch size, enc_hid_dim * 2]

    hidden = torch.tanh(self.fc(concatenated))
    # hidden -> [batch size, dec_hid_him]

    return outputs, hidden
    

In [18]:
class Attention(torch.nn.Module):

  def __init__(self, enc_hid_dim, dec_hid_dim):
    super().__init__()
    self.attn = torch.nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
    self.v = torch.nn.Linear(dec_hid_dim, 1, bias=False)

  def forward(self, encoder_outputs, hidden):

    # encoder_outputs -> [src len, batch size, enc_hid_dim * 2]
    # hidden -> [batch size, dec_hid_dim]

    src_len = encoder_outputs.shape[0]
    batch_size = encoder_outputs.shape[1]

    # Repeat hidden state for each time step in src len
    hidden = hidden.unsqueeze(0).repeat(src_len, 1, 1)
    # hidden -> [src len, batch size, dec_hid_dim]

    concatenated = torch.cat((encoder_outputs, hidden), dim=2)
    # concatenated -> [src len, batch size, (enc_hid_dim * 2) + dec_hid_dim]

    energy = torch.tanh(self.attn(concatenated))
    # energy -> [src len, batch size, dec_hid_dim]

    attention = self.v(energy).squeeze(2)
    # attention -> [src len, batch size]

    return F.softmax(attention, dim=0)
    # return dim -> [src len, batch size]
    # the `src len` values of each item in batch add up to 1 (because of softmax)


In [19]:
class Decoder(torch.nn.Module):

  def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
    super().__init__()

    self.output_dim = output_dim

    self.embedding = torch.nn.Embedding(output_dim, emb_dim)
    self.rnn = torch.nn.GRU(emb_dim + (2 * enc_hid_dim), dec_hid_dim)
    self.fc = torch.nn.Linear(emb_dim + dec_hid_dim + (2 * enc_hid_dim), output_dim)
    self.dropout = torch.nn.Dropout(dropout)
    self.attn = attention

  def forward(self, input, encoder_outputs, hidden):
    
    # input -> [batch size]
    # encoder_outputs -> [src len, batch size, enc_hid_dim * 2]
    # hidden -> [batch size, dec_hid_dim]

    input = input.unsqueeze(0)
    # input -> [1, batch size]

    embedded = self.dropout(self.embedding(input))
    # embedded -> [1, batch size, emb_dim]

    a = self.attn(encoder_outputs, hidden)
    # a -> [src len, batch size]

    encoder_outputs = encoder_outputs.permute(1, 0, 2)
    # encoder_outputs -> [batch size, src len, enc_hid_dim * 2]

    a = a.permute(1, 0).unsqueeze(1)
    # a -> [batch size, 1, src len]

    weighted = torch.bmm(a, encoder_outputs).permute(1, 0, 2)
    # weighted -> [1, batch size, enc_hid_dim * 2]

    outputs, hidden = self.rnn(torch.cat((embedded, weighted), dim=2), hidden.unsqueeze(0))
    # outputs -> [1, batch size, dec_hid_dim]
    # hidden -> [1, batch size, dec_hid_dim]

    fc_input = torch.cat((embedded, outputs, weighted), dim=2)
    # fc_input -> [1, batch size, emb_dim + dec_hid_dim + (2 * enc_hid_dim)]

    predictions = self.fc(fc_input).squeeze(0)
    # predictions -> [batch size, output_dim]

    return predictions, hidden.squeeze(0)
    # return dimensions
    # predictions -> [batch size, output_dim]
    # hidden -> [batch size, dec_hid_dim]


In [20]:
class Seq2Seq(torch.nn.Module):

  def __init__(self, encoder, decoder, device):
    super().__init__()

    self.encoder = encoder
    self.decoder = decoder
    self.device = device

  def forward(self, src, trg, teacher_forcing_ratio=0.5):

    # src -> [src len, batch size]
    # trg -> [trg len, batch size]

    batch_size = trg.shape[1]
    trg_len = trg.shape[0]
    trg_vocab_size = self.decoder.output_dim

    # decoder outputs
    outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

    # pass src through encoder
    encoder_outputs, hidden = self.encoder(src)
    # encoder_outputs -> [src len, batch size, enc_hid_dim * 2]
    # hidden -> [batch size, dec_hid_him]

    input = trg[0, :]
    # input -> [batch size]

    for t in range(1, trg_len):

      output, hidden = self.decoder(input, encoder_outputs, hidden)
      # output -> [batch size, output_dim]
      # hidden -> [batch size, dec_hid_dim]

      outputs[t] = output

      teacher_force = random.random() < teacher_forcing_ratio

      top1 = output.argmax(1)
      # top1 -> [batch size]

      input = trg[t] if teacher_force else top1

    return outputs
    


In [21]:
def init_weights(m):
  for name, param in m.named_parameters():
    if 'weight' in name:
      torch.nn.init.normal_(param.data, mean=0, std=0.01)
    else:
      torch.nn.init.constant_(param.data, 0)

def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

In [22]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
ENC_HID_DIM = 128
DEC_HID_HIM = 128

DROPOUT = 0.2

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_HIM, DROPOUT)
attn = Attention(ENC_HID_DIM, DEC_HID_HIM)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_HIM, DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)
model.apply(init_weights)

print(f'Model parameters: {count_parameters(model):,}')

Model parameters: 6,267,183


In [23]:
model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(5893, 128)
    (rnn): GRU(128, 128, bidirectional=True)
    (fc): Linear(in_features=256, out_features=128, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(7855, 128)
    (rnn): GRU(384, 128)
    (fc): Linear(in_features=512, out_features=7855, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
    (attn): Attention(
      (attn): Linear(in_features=384, out_features=128, bias=True)
      (v): Linear(in_features=128, out_features=1, bias=False)
    )
  )
)

In [24]:
optimizer = optim.Adam(model.parameters())

In [25]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = torch.nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [26]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in tqdm(enumerate(iterator), total=len(iterator)):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [27]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in tqdm(enumerate(iterator), total=len(iterator)):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [28]:
%%writefile gpu_usage.sh
#! /bin/bash
#comment: run for 60 seconds, change it as per your use
end=$((SECONDS+60))

while [ $SECONDS -lt $end ]; do
    nvidia-smi --format=csv --query-gpu=power.draw,utilization.gpu,memory.used,memory.free,fan.speed,temperature.gpu >> gpu.log
    #comment: or use below command and comment above using #
    #nvidia-smi dmon -i 0 -s mu -d 1 -o TD >> gpu.log
done

Writing gpu_usage.sh


In [29]:
%%bash --bg

bash gpu_usage.sh

Starting job # 0 in a separate thread.


In [30]:
N_EPOCHS = 25
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')


Epoch: 01 | Time: 0m 39s
	Train Loss: 5.916 | Train PPL: 370.768
	 Val. Loss: 5.211 |  Val. PPL: 183.257



Epoch: 02 | Time: 0m 38s
	Train Loss: 4.857 | Train PPL: 128.617
	 Val. Loss: 5.112 |  Val. PPL: 165.945



Epoch: 03 | Time: 0m 38s
	Train Loss: 4.536 | Train PPL:  93.318
	 Val. Loss: 5.060 |  Val. PPL: 157.622



Epoch: 04 | Time: 0m 38s
	Train Loss: 4.400 | Train PPL:  81.417
	 Val. Loss: 5.114 |  Val. PPL: 166.377



Epoch: 05 | Time: 0m 38s
	Train Loss: 4.312 | Train PPL:  74.606
	 Val. Loss: 5.217 |  Val. PPL: 184.461


...


Epoch: 21 | Time: 0m 38s
	Train Loss: 2.091 | Train PPL:   8.096
	 Val. Loss: 3.185 |  Val. PPL:  24.167



Epoch: 22 | Time: 0m 38s
	Train Loss: 2.020 | Train PPL:   7.538
	 Val. Loss: 3.157 |  Val. PPL:  23.506



Epoch: 23 | Time: 0m 38s
	Train Loss: 1.958 | Train PPL:   7.083
	 Val. Loss: 3.168 |  Val. PPL:  23.755



Epoch: 24 | Time: 0m 38s
	Train Loss: 1.844 | Train PPL:   6.320
	 Val. Loss: 3.171 |  Val. PPL:  23.837



Epoch: 25 | Time: 0m 38s
	Train Loss: 1.777 | Train PPL:   5.911
	 Val. Loss: 3.160 |  Val. PPL:  23.579


In [31]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} |  Test PPL: {math.exp(test_loss):7.3f} |')


| Test Loss: 3.134 |  Test PPL:  22.965 |


In [32]:
def display_attention(sentence, translation, attention):

  fig = plt.figure(figsize=(10, 10))
  ax = fig.add_subplot(111)

  attention = attention.squeeze(1).cpu().detach().numpy()

  cax = ax.matshow(attention, cmap='bone')

  ax.tick_params(labelsize=15)
  ax.set_xticklabels(['']+['<sos>']+[t.lower() for t in sentence]+['<eos>'], rotation=45)
  ax.set_yticklabels(['']+translation)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()
  plt.close()

In [33]:
example_idx = 12

src = vars(train_data.examples[example_idx])['src']
trg = vars(train_data.examples[example_idx])['trg']

print(f'src: ${src}')
print(f'trg: ${trg}')

src: $['a', 'black', 'dog', 'and', 'a', 'spotted', 'dog', 'are', 'fighting']
trg: $['ein', 'schwarzer', 'hund', 'und', 'ein', 'gefleckter', 'hund', 'kämpfen', '.']


In [34]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len=50):

  model.eval()

  if isinstance(sentence, str):
    nlp = spacy.load('en')
    tokens = [token.text.lower() for token in nlp(sentence)]
  else:
    tokens = [token.lower() for token in sentence]

  tokens = [src_field.init_token] + tokens + [src_field.eos_token]

  src_indexes = [src_field.vocab.stoi[token] for token in tokens]

  src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

  with torch.no_grad():
    encoder_outputs, hidden = model.encoder(src_tensor)

  trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

  for i in range(max_len):

    trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

    with torch.no_grad():
      output, hidden = model.decoder(trg_tensor, encoder_outputs, hidden)

      pred_token = output.argmax(1).item()

      trg_indexes.append(pred_token)

      if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
        break

  trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

  return trg_tokens

In [35]:
translate_sentence(src, SRC, TRG, model, device)

['<sos>',
 'ein',
 'schwarzer',
 'hund',
 'und',
 'ein',
 'gefleckter',
 'hund',
 'kämpfen',
 'um',
 'ein',
 'hund',
 'kämpfen',
 '.',
 '<eos>']

In [36]:
from torchtext.data.metrics import bleu_score

def calculate_bleu(data, src_field, trg_field, model, device, max_len = 50):

  trgs = []
  pred_trgs = []

  for datum in data:

    src = vars(datum)['src']
    trg = vars(datum)['trg']

    pred_trg = translate_sentence(src, SRC, TRG, model, device, max_len)

    pred_trg = pred_trg[:-1]

    pred_trgs.append(pred_trg)
    trgs.append([trg])

  return bleu_score(pred_trgs, trgs)

bleu_score = calculate_bleu(test_data, SRC, TRG, model, device)

print(f'BLEU score = {bleu_score*100:.2f}')

BLEU score = 23.85
